# NLP with LSTM Model
|Title|Link|
|-|-|
|Zipf's Law|https://github.com/Ashvith/Zipf-s-Law/blob/master/zipfs_law.ipynb|
|Next word prediction with NLP and deep learning|https://towardsdatascience.com/next-word-prediction-with-nlp-and-deep-learning-48b9fe0a17bf|

### Import the libraries

In [ ]:
# Inbuilt libraries, NLTK and pandas
import pickle
import numpy as np
import os
import nltk
import pandas as pd     
# from nltk.corpus import stopwords
from matplotlib import pyplot as plt

# Tensorflow and Keras libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow import keras
import os
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"]="true"


### Extracting all the words in the corpus
  
>If you're working with LSTM’s or other models which capture the semantic meaning and the meaning of a word depends on the context of the previous text, then it becomes important not to remove stopwords.

In [ ]:
words_doc = nltk.Text(nltk.corpus.gutenberg.words('austen-emma.txt'))
# stop_words = set(stopwords.words('english'))
# nltk.corpus.gutenberg.fileids()


### Converting to lower case
>Again, we will NOT filter the stopwords out for the same reason mentioned above

In [ ]:
data= [word.lower() for word in words_doc if word.isalpha()]
data = ' '.join(data)
# words_doc = [word for word in words_doc if word not in stop_words]

In [ ]:
print(data)

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])

pickle.dump(tokenizer, open('tokenizer1.pkl', 'wb'))

sequence_data = tokenizer.texts_to_sequences([data])[0]

In [ ]:
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

In [ ]:
sequences = []

for i in range(1, len(sequence_data)):
    words = sequence_data[i-1:i+1]
    sequences.append(words)

print("The length of sequences are: ", len(sequences))
sequences = np.array(sequences)
sequences[:10]

In [ ]:
x_train = []
y_train = []

for i in sequences:
    x_train.append(i[0])
    y_train.append(i[1])

x_train = np.array(x_train)
y_train = np.array(y_train)

In [ ]:
print("The data is:", x_train[:5])
print("The responses are:", y_train[:5])

In [ ]:
y_train = to_categorical(y_train, num_classes=vocab_size)
y_train[:5]

In [ ]:
model = Sequential([
    Embedding(vocab_size, 5, input_length=1),
    LSTM(512, return_sequences=True),
    LSTM(512),
    Dense(512, activation="relu"),
    Dense(vocab_size, activation="softmax")
])

In [ ]:
model.summary()

In [ ]:
from keras.utils.vis_utils import plot_model
keras.utils.plot_model(model, to_file='model.png', show_layer_names=True)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import TensorBoard

checkpoint =ModelCheckpoint("nextword1.h5", monitor='loss', verbose=1, save_best_only=True, mode='auto')

reduce = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=3, min_lr=0.001, verbose = 1)

logdir='logsnextword1'
tensorboard_Visualization = TensorBoard(log_dir=logdir)

In [ ]:
model.compile(loss="categorical_crossentropy", optimizer=Adam(lr=0.0001), metrics="accuracy")

In [ ]:
model.fit(x_train, y_train, epochs=150, batch_size=64, callbacks=[checkpoint, reduce, tensorboard_Visualization])